# Data Generation Through Workflow in Synthora

Workflow is a powerful system in Synthora that can be used to orchestrate 
agents on solving various problems, allowing users to define their own workflow
with a high level of flexibility.

In this tutorial, we will show you how to use workflow to generate data. We
will start from the simplest SFT data generation and gradually increase the
complexity to COT and ToT data generation. Thanks to the flexibility of
workflow, the overall process is simple and can be easily customized.

Now, if you are ready, let's start!

## Prerequisites
Before jumping into the fun stuff, there are a few things you’ll need to set up. (Hang tight—it’s worth it!)

### Install Synthora
Synthora runs on Python 3.8 or later. You can install it quickly using pip:

In [10]:
%pip install synthora

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Packages & Set Your API Key

In this tutorial, we’ll be using OpenAI’s API for data generation. Before we proceed, let’s import the necessary packages and configure the API key:

In [11]:
import os
import textwrap
from getpass import getpass
from typing import Any, Dict, List

from synthora.agents import VanillaAgent
from synthora.agents.tot_agent import ToTAgent
from synthora.messages import user
from synthora.messages.base import BaseMessage
from synthora.prompts.buildin import ZeroShotCoTPrompt
from synthora.utils.pydantic_model import get_pydantic_model
from synthora.workflows import task
from synthora.workflows.base_task import BaseTask
from synthora.workflows.scheduler.process_pool import ProcessPoolScheduler
from synthora.workflows.scheduler.thread_pool import ThreadPoolScheduler

In [12]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key here: ")

### Prompt Preparation

To make comparisons easier, let’s prepare prompts for generating data. A good prompt is the foundation for successful data generation. Take time to think through the kind of data you need and craft your prompt accordingly.

In [13]:
problems = [
    "How many letters 'r' in the word 'strawberry'?",
    "9.11 and 9.9, which one is bigger?",
]

## Simple SFT Data Generation

Now we can start to get our hands dirty (Finally!). First, we define a task to
generate data using a vanilla agent, which is basically the simplest and most
basic, classical form of agents.

We first define two tasks, one of which is to generate data using a vanilla
agent (basically just run a query), and the other is to convert the data to
the format we want.

> What are tasks in Synthora?
>
> Tasks are basic units of workload in workflow. They are the smallest units
> of work that can be executed independently. In Synthora, tasks are defined
> using the `@task` decorator. For more details, please refer to our official
> documentation of workflow.


In [14]:
@task
def generate_simple_data(prompt: str) -> List[BaseMessage]:
    agent = VanillaAgent.default()
    _ = agent.run(prompt)
    return agent.history


@task
def format_data(*resps: List[BaseMessage]) -> List[Dict[str, str]]:
    return [
        {
            "prompt": str(resp[0].content),
            "instruct": str(resp[1].content),
            "response": str(resp[2].content),
        }
        for resp in resps
    ]

Then we can define a workflow to first generate data on each question and
format the responses into a readable format like below:

In [15]:
flow = ThreadPoolScheduler.map(generate_simple_data, problems) >> format_data
flow.run()

[{'prompt': '\nYou are an AI assistant.\n',
  'instruct': "How many letters 'r' in the word 'strawberry'?",
  'response': 'The word "strawberry" has two letters \'r\'.'},
 {'prompt': '\nYou are an AI assistant.\n',
  'instruct': '9.11 and 9.9, which one is bigger?',
  'response': 'The number 9.11 is bigger than 9.9. When comparing the two decimals after the decimal point, 11 is greater than 9.'}]

There we get some simple data that can be used for simple SFT, which is
basically just some query and answers without any intermediate steps. The
quality of the data, however, could be low and containing mistakes.

Now, what if we want to generate more complex data with better quality? It's
time for Chain of Thoughts (COT) or Tree of Thoughts (ToT) data to kick in.

## CoT Data Generation

Well, it's not that hard to generate CoT data actually. The only thing we need
to do is to change the prompt to `ZeroShotCoTPrompt`, which is the simplest
prompt guiding the agent to output the thinking steps.

> We are using `ZeroShotCoTPrompt`, which is a basical CoT prompts with no 
> examples, for a quick glance here. If you want to generate data with higher 
> quality (or more at your preference), you can augment it with some examples 
> of your own.

In [16]:
@task
def generate_cot_data(prompt: str) -> List[BaseMessage]:
    agent = VanillaAgent.default(ZeroShotCoTPrompt)
    _ = agent.run(prompt)
    return agent.history


flow = ThreadPoolScheduler.map(generate_cot_data, problems) >> format_data
flow.run()

[{'prompt': '\nSolve the following problem step by step. For each step,\ncarefully explain your reasoning, include all calculations, and state any assumptions you make.\nEnsure that each step logically leads to the next, and provide a clear and concise final answer at the end.\nIf relevant, break the problem into smaller parts and address each part individually before combining the results.\n',
  'instruct': "How many letters 'r' in the word 'strawberry'?",
  'response': 'To solve the problem of finding how many times the letter \'r\' appears in the word \'strawberry\', we can follow these steps:\n\n1. **Identify the word:** We are working with the word "strawberry".\n\n2. **Look at each letter individually:** \n   - The first letter is \'s\'.\n   - The second letter is \'t\'.\n   - The third letter is \'r\'.\n   - The fourth letter is \'a\'.\n   - The fifth letter is \'w\'.\n   - The sixth letter is \'b\'.\n   - The seventh letter is \'e\'.\n   - The eighth letter is \'r\'.\n   - The 

Nice, we just got some data containing steps of thinking, which appearently
has better quality compared with our first version.

This is not the end, however. Sometimes a single CoT process won't solve the 
question we gave. To address this, we can use Tree of Thoughts (ToT).

## ToT Data Generation

ToT data generation will apply the following procedure:

1. For each step, the agent will generate multiple answers.
2. Another agent will search through the tree by BFS or DFS to check if there
   exists a path where the problem has been solved successfully.
   
ToT will usually improve the success rate of problem solving, and also improve
the quality of the data generated. Unfortunately, since ToT applies a new 
approach of data generation, it won't be as that easy as CoT, which can be 
simply done by altering the prompt. But no worries! We got your back. 

We offer a `ToTAgent` in Synthora, which encapsulates all the dirty works for
users. In `ToTAgent`, the tree will be searched with DFS, and we only need to 
make some configurations like `level_size` or `max_turns` here.

In [17]:
@task
def generate_tot_data(prompt: str) -> List[BaseMessage]:
    agent = ToTAgent.default(level_size=2, max_turns=15)
    resp = agent.run(prompt)
    if resp.is_err:
        # the problem is not solved successfully
        return []
    return agent.history

Then we can create a even harder question for the agent to solve.

In [18]:
hard_question = (
    "Consider a regular octagon. How many different triangles can be formed "
    "if the octagon is placed inside a circle and we can also use the center "
    "of the circle as a vertex for the triangles? Let's think step by step."
)
flow = ThreadPoolScheduler.map(generate_tot_data, problems + [hard_question])
results = flow.run()

# Get the data for the last question
for res in results[-1][1:]:
    print(res.content)

Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step.
To solve this problem, let's break it down into manageable steps, considering both the properties of a regular octagon and the addition of the circle's center as a potential vertex for forming triangles.

### Step 1: Understand and Define the Components Available for Triangle Formation

**Think**: A regular octagon has 8 vertices. Additionally, we have the center of the circle as another point to consider. We need to calculate how many distinct triangles can be formed using any three of these nine points (8 vertices of the octagon + 1 center point of the circle).

**Action**: Define our points as follows:
- Let \( V_1, V_2, \ldots, V_8 \) be the vertices of the octagon.
- Let \( C \) be the center of the circle.

Now, we need to categorize possible triangles based on whether 

We can see that the hard problem has been solved successfully. We can also
check the data generated for the question comparing 9.11 and 9.9, just for
comparison with previous approaches like CoT.

In [19]:
for res in results[1][1:]:
    print(res.content)

9.11 and 9.9, which one is bigger?
To determine which of the two numbers, 9.11 and 9.9, is bigger, we should compare their values. Let's do this step by step.

**Think**: We start by comparing each digit of both numbers from left to right. This approach will help us accurately establish which number is greater without direct calculation.

1. **First digit**: Both numbers have the digit '9' in the units (integer) place. Since they are equal, we move to the next digit.

2. **Second digit**: In the tenths place, both numbers again have '9'. Since these are also equal, we will proceed to check the next digit.

3. **Third digit**: For 9.11, the next digit in the hundredths place is '1'. The number 9.9 does not have a digit in this place, which can alternatively be written as 9.90 where the hundredths place is '0'.

Therefore, comparing the third digit, '1' (in 9.11) is greater than '0' (in 9.9 or 9.90).

**Output**: 9.11 is greater than 9.9 based on the comparison of digits in their decimal

## Another Example: Scoring Generated Data

Now, I believe you already have a brief sense on how to use Synthora to
generate simple, CoT and ToT data. At the end of this tutorial, we gonna walk
through another case, where we will generate multiple entries of data on the
same problem and let one agent to score each of them.

First we can define a function used to score two entries of generated data:

In [20]:
def score_response(
    history1: List[BaseMessage], history2: List[BaseMessage], prompt: str
) -> Dict[str, Any]:
    response_format = get_pydantic_model('{"score1": 0.0, "score2": 0.0}')

    system_prompt = textwrap.dedent(
        f"""\
        You are a judge to score responses to the following question, scaling from 0 to 10.

        {prompt}
        """  # noqa: E501
    )
    agent = VanillaAgent.default(system_prompt)
    agent.model.config["response_format"] = response_format

    # Skip the system message in history
    openai_history1 = [msg.to_openai_message() for msg in history1[1:]]
    openai_history2 = [msg.to_openai_message() for msg in history2[1:]]

    _history1 = "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in openai_history1]
    )
    _history2 = "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in openai_history2]
    )

    agent.history.append(user(f"Response 1:\n{_history1}"))
    agent.history.append(user(f"Response 2:\n{_history2}"))
    resp = agent.run("Please score the two responses.").unwrap().parsed
    result = {
        "chosen": openai_history1
        if resp.score1 > resp.score2
        else openai_history2,
        "rejected": openai_history2
        if resp.score1 > resp.score2
        else openai_history1,
        "score_chosen": resp.score1
        if resp.score1 > resp.score2
        else resp.score2,
        "score_rejected": resp.score2
        if resp.score1 > resp.score2
        else resp.score1,
    }
    return result

Then we can define a function of generating two entries of data with the 
workflow like below, where two tasks (agents) will run concurrently trying to
solve the same problem.

In [21]:
def generate_data(system1: str, system2: str, prompt: str) -> Dict[str, Any]:
    agent1, agent2 = (
        VanillaAgent.default(system1),
        VanillaAgent.default(system2),
    )
    flow = (BaseTask(agent1.run) | BaseTask(agent2.run)).s(prompt)
    _ = flow.run()
    return score_response(agent1.history, agent2.history, prompt)

For each problem, we can have a system message for it. For the consideration 
of convenience, we will use the same simple message for each problem.

In [22]:
system_message = "You are an AI Assistant."
system1 = [system_message for _ in problems]
system2 = [system_message for _ in problems]

Then we can run all the problems concurrently with `ProcessPoolScheduler`, 
which can run tasks with multi-process approach.

> Here we have two nested parallelism, which is supported by nested workflow:
> 
> 1. The outmost workflow works with multi-process, where each problem takes
>   up a process to be solved.
> 2. Inside each process, there will also be a workflow, where two agents are
>   trying to solve the same problem concurrently with multi-thread.

In [23]:
flow = ProcessPoolScheduler.starmap(
    BaseTask(generate_data), zip(system1, system2, problems)
)
results = flow.run()

RemoteError: 
---------------------------------------------------------------------------
Traceback (most recent call last):
  File "D:\Software\miniconda3\envs\syntropix\Lib\multiprocessing\managers.py", line 253, in serve_client
    request = recv()
              ^^^^^^
  File "D:\Software\miniconda3\envs\syntropix\Lib\multiprocessing\connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'generate_data' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
---------------------------------------------------------------------------

Then we can print the result to see the accepted and rejected data to each
problem, according to the score:

In [14]:
for result in results:
    print(result)

{'chosen': [{'content': "How many letters 'r' in the word 'strawberry'?", 'role': 'user', 'name': 'user'}, {'content': "The word 'strawberry' contains three letters 'r'.", 'role': 'assistant', 'name': 'gpt-4o'}], 'rejected': [{'content': "How many letters 'r' in the word 'strawberry'?", 'role': 'user', 'name': 'user'}, {'content': 'The word "strawberry" contains two letters \'r\'.', 'role': 'assistant', 'name': 'gpt-4o'}], 'score_chosen': 10.0, 'score_rejected': 6.0}
{'chosen': [{'content': '9.11 and 9.9, which one is bigger?', 'role': 'user', 'name': 'user'}, {'content': '9.11 is larger than 9.9. This is because 9.9 is equivalent to 9.90, and when comparing the numbers 9.11 and 9.90, the first decimal place is the same (9), but the second decimal place shows that 11 is greater than 9.', 'role': 'assistant', 'name': 'gpt-4o'}], 'rejected': [{'content': '9.11 and 9.9, which one is bigger?', 'role': 'user', 'name': 'user'}, {'content': 'The number 9.11 is bigger than 9.9. To compare the 

## Highlights

In this tutorial, we walked through the generation of simple SFT data, CoT
data, and ToT data, with the support of workflow in Synthora. At last, we
introduced an example, where we can generate multiple entries of data 
concurrently, taking leverage of the support of nested workflow.
 

## About Synthora

Synthora is a lightweight and extensible framework for LLM-driven Agents and 
ALM research. It provides essential components to build, test and evaluate 
agents. At its core, Synthora aims to assemble an agent with a single config, 
thus minimizing your effort in building, tuning, and sharing agents.

If you find this tutorial interesting, feel free to visit our
[GitHub Repo](https://github.com/syntropix-ai/synthora) and leave a star🌟!
Any feedback from you will mean a lot to us.